In [10]:
import pandas as pd
import numpy as np
import os
from scipy.stats import skew, kurtosis, entropy, iqr
from scipy.fft import fft, fftfreq
from scipy.signal import welch, find_peaks
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
from IPython.display import display, HTML

# === Part 1: Feature Engineering ===

# Suppress matplotlib warnings
warnings.filterwarnings("ignore", category=UserWarning, module='matplotlib')

# Configuration
folder_path = "./harth"  
sensor_columns = ['back_x', 'thigh_x']  # Only analyzing these
sampling_interval = 0.02  # 50 Hz


In [11]:
# Data Aggregation for Visualization (only back_x)
all_sensor_values = []
all_timestamps = []

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            if 'timestamp' not in df.columns or 'back_x' not in df.columns:
                continue
            timestamps = pd.to_datetime(df['timestamp'], errors='coerce')
            sensor_data = df['back_x']
            valid = timestamps.notna() & sensor_data.notna()
            timestamps = timestamps[valid].to_numpy()
            sensor_data = sensor_data[valid].to_numpy()
            if len(sensor_data) < 2:
                continue
            all_sensor_values.extend(sensor_data)
            all_timestamps.extend(timestamps)
        except:
            continue

x_viz = np.array(all_sensor_values)
N_viz = len(x_viz)


In [12]:
# Feature Extraction for back_x and thigh_x
features = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        for sensor_column in sensor_columns:
            if sensor_column not in df.columns:
                continue

            sensor_data = df[sensor_column].dropna().to_numpy()
            if len(sensor_data) < 2:
                continue

            x = sensor_data
            N = len(x)

            # --- Time Domain Features ---
            features.update({
                f'{sensor_column}_Mean': np.mean(x),
                f'{sensor_column}_Std': np.std(x),
                f'{sensor_column}_Var': np.var(x),
                f'{sensor_column}_Min': np.min(x),
                f'{sensor_column}_Max': np.max(x),
                f'{sensor_column}_Range': np.ptp(x),
                f'{sensor_column}_Median': np.median(x),
                f'{sensor_column}_IQR': iqr(x),
                f'{sensor_column}_RMS': np.sqrt(np.mean(x**2)),
                f'{sensor_column}_ZCR': ((x[:-1] * x[1:]) < 0).sum(),
                f'{sensor_column}_Skew': skew(x),
                f'{sensor_column}_Kurtosis': kurtosis(x),
                f'{sensor_column}_Energy': np.sum(x**2),
                f'{sensor_column}_AC_lag1': np.corrcoef(x[:-1], x[1:])[0, 1] if len(x) > 1 else np.nan,
                f'{sensor_column}_Peak_Count': len(find_peaks(x)[0]),
                f'{sensor_column}_Peak_Amplitude': np.max(x[find_peaks(x)[0]]) if len(find_peaks(x)[0]) > 0 else np.nan,
            })

            # --- Frequency Domain Features ---
            yf = fft(x)
            xf = fftfreq(N, sampling_interval)[:N // 2]
            amplitudes = 2.0 / N * np.abs(yf[0:N // 2])
            f_psd, psd_values = welch(x, fs=1 / sampling_interval)

            features.update({
                f'{sensor_column}_Spectral_Centroid': np.sum(xf * amplitudes) / np.sum(amplitudes),
                f'{sensor_column}_Spectral_Energy': np.sum(amplitudes**2),
                f'{sensor_column}_Spectral_Entropy': entropy(amplitudes / np.sum(amplitudes)),
                f'{sensor_column}_Freq_Var': np.var(amplitudes),
                f'{sensor_column}_Spectral_Flatness': np.exp(np.mean(np.log(amplitudes + 1e-10))) / np.mean(amplitudes),
                f'{sensor_column}_Peak_Freq': xf[np.argmax(amplitudes)],
                f'{sensor_column}_Bandwidth': np.max(xf) - np.min(xf),
                f'{sensor_column}_PSD_Mean': np.mean(psd_values),
                f'{sensor_column}_PSD_Max': np.max(psd_values),
            })

            # Flatten FFT Coefficients
            for i, val in enumerate(amplitudes[:10]):
                features[f'{sensor_column}_FFT_Coeff_{i+1}'] = val
        break  # Just use first valid file for Part 1


In [ ]:
# Create DataFrame and Display Table
features_df = pd.DataFrame([features]).round(4)
html_output = features_df.to_html(index=False, max_cols=None, escape=False)
display(HTML(f"<div style='overflow-x:auto'>{html_output}</div>"))

# Time Domain Plot (back_x only)
plt.figure(figsize=(14, 5))
plt.plot(all_timestamps[:1000], x_viz[:1000], color='darkorange', linewidth=1.8)
plt.gca().set_facecolor('#f7f7f7')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.xticks(rotation=30)
plt.title("Time Domain Signal — back_x", fontsize=14, fontweight='bold')
plt.xlabel("Timestamp")
plt.ylabel("Sensor Value")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Frequency Domain Plot (back_x only)
yf_viz = fft(x_viz)
xf_viz = fftfreq(N_viz, sampling_interval)[:N_viz // 2]
amplitudes_viz = 2.0 / N_viz * np.abs(yf_viz[0:N_viz // 2])

plt.figure(figsize=(14, 5))
plt.plot(xf_viz, amplitudes_viz, color='royalblue', linewidth=1.5)
plt.gca().set_facecolor('#f2f4f8')
plt.yscale('log')
plt.title("Frequency Spectrum (FFT) — back_x", fontsize=14, fontweight='bold')
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude (log scale)")
plt.grid(True, which='both', linestyle='--', alpha=0.4)

# Annotate peak
peak_index = np.argmax(amplitudes_viz)
peak_freq = xf_viz[peak_index]
peak_amp = amplitudes_viz[peak_index]
plt.annotate(f"Peak: {peak_freq:.2f} Hz",
             xy=(peak_freq, peak_amp),
             xytext=(peak_freq + 2, peak_amp),
             arrowprops=dict(arrowstyle="->", color='gray'),
             fontsize=10)
plt.tight_layout()
plt.show()


back_x_Mean,back_x_Std,back_x_Var,back_x_Min,back_x_Max,back_x_Range,back_x_Median,back_x_IQR,back_x_RMS,back_x_ZCR,back_x_Skew,back_x_Kurtosis,back_x_Energy,back_x_AC_lag1,back_x_Peak_Count,back_x_Peak_Amplitude,back_x_Spectral_Centroid,back_x_Spectral_Energy,back_x_Spectral_Entropy,back_x_Freq_Var,back_x_Spectral_Flatness,back_x_Peak_Freq,back_x_Bandwidth,back_x_FFT_Coeffs,back_x_PSD_Mean,back_x_PSD_Max,thigh_x_Mean,thigh_x_Std,thigh_x_Var,thigh_x_Min,thigh_x_Max,thigh_x_Range,thigh_x_Median,thigh_x_IQR,thigh_x_RMS,thigh_x_ZCR,thigh_x_Skew,thigh_x_Kurtosis,thigh_x_Energy,thigh_x_AC_lag1,thigh_x_Peak_Count,thigh_x_Peak_Amplitude,thigh_x_Spectral_Centroid,thigh_x_Spectral_Energy,thigh_x_Spectral_Entropy,thigh_x_Freq_Var,thigh_x_Spectral_Flatness,thigh_x_Peak_Freq,thigh_x_Bandwidth,thigh_x_FFT_Coeffs,thigh_x_PSD_Mean,thigh_x_PSD_Max
-0.9303,0.5758,0.3316,-8.0,1.356,9.356,-0.9023,0.6584,1.0941,2125,-1.6403,12.5347,213933.7806,0.6668,29749,1.356,10.315,4.1251,10.9053,0.0,0.6911,0.0,24.9997,"[1.8606243853152489, 0.07838764039408867, 0.06159749961683046, 0.03582106602129395, 0.036284009481161915, 0.046135370373916615, 0.055652345695209986, 0.05033109812751146, 0.04828848708203648, 0.052921237394887584]",0.012,0.3042,-1.0428,1.041,1.0836,-8.0,7.1904,15.1904,-0.9844,0.939,1.4735,16476,-0.9203,5.0679,388016.2021,0.3949,32715,7.1904,10.8153,6.5172,11.0301,0.0001,0.7047,0.0,24.9997,"[2.0856584893350343, 0.17585745353428076, 0.12721629534512136, 0.06317462413779375, 0.09977964825610684, 0.11750484720917015, 0.1085054742413855, 0.09011899970544886, 0.07297472365482602, 0.07412662933919913]",0.0403,0.3189


NameError: name 'x_viz' is not defined

<Figure size 1400x500 with 0 Axes>